# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [19]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [20]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):

# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [21]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [22]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [23]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [24]:
#Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [25]:
#Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [26]:
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
#artist ,song title ,song length ,when sessionid=338, iteminsession=4 ==>primary key
#we need to create table based on the query we want to do
#colums
# 1-artist name
# 2-song title 
# 3-length
# 4-session id 
# 5-item in  session
query="Create Table if Not EXISTS  music_info_session "
query=query+ " (session_id int ,item_nsession int ,   artistname text,    song_title text ,song_length float  ,primary key (session_id, item_nsession))"

try:
    session.execute(query)
except  Exception as p:
    print(p)
    

In [27]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #print((line) )
#the INSERT statements into the `query` variable
        query = "insert into music_info_session  (session_id ,artistname ,song_title  ,song_length  ,item_nsession)  "
        query = query + "values (%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), line[0] ,line[9], float(line[5]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

in this query we are gonna fetch the artistname ,song title and song's length  by sessionid and iteminsession.our primary key will be composite of two column :: sessionid ,iteminsession.


In [28]:
#Add in the SELECT statement to verify the data was entered into the table


try:
    rows = session.execute("select session_id,item_nsession,artistname ,song_length from music_info_session  where session_id = 338 ")
except Exception as e:
    print(e)

for row in rows:
    #print(row)
    
    print (row.session_id,"||" ,row.artistname, "||",  row.song_length,  "||", row.item_nsession)

338 || Pixies || 89.36444091796875 || 1
338 || The Roots / Jack Davey || 155.95057678222656 || 2
338 || Mike And The Mechanics || 275.1211853027344 || 3
338 || Faithless || 495.30731201171875 || 4


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [29]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
#userid  sessionid artist song (sorted by itemInSession first and last name
query ="create Table If not EXISTS  artist_liberay"
query=query+"""(userid int , sessionid int , itemInSession int, artist_name  text,song_name text,\
first_name text,last_name Text,\
PRIMARY KEY ((userid,sessionid) ,itemInSession ))"""
#itemin session do it as clustring and for sorting only purpose
try :
    session.execute(query)
except  Exception as p:
    print(p)

In [30]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #print((line) )
#the INSERT statements into the `query` variable
        query = "insert into artist_liberay (userid ,sessionid ,itemInSession  ,artist_name  ,song_name ,first_name ,last_name )  "
        query = query + "values (%s     ,%s        ,%s             ,%s           ,%s        ,%s         ,%s)"
        ##  column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
     

this query to get the name of the artist ,song sorted iteminSession and user first name ,second name 
by using composite primary key::  partiion key as  (user id ,session id) and cluster colum iteminSession


### testing the  out put of the query 2

In [31]:

try:
    rows = session.execute("select artist_name,song_name ,first_name,last_name from artist_liberay  where userid = 10 and sessionid = 182 ")
except Exception as e:
    print(e)

for row in rows:
    #print(row)
    
    print (row.artist_name,"||" , row.song_name,"||" ,row.first_name,"||" ,row.last_name)

Down To The Bone || Keep On Keepin' On || Sylvie || Cruz
Three Drives || Greece 2000 || Sylvie || Cruz
Sebastien Tellier || Kilometer || Sylvie || Cruz
Lonnie Gordon || Catch You Baby (Steve Pitron & Max Sanna Radio Edit) || Sylvie || Cruz


In [32]:
#Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
# by song will be primary key 
query="create table if not exists user_history"
query=query+"(song_name text,userid int,artist_name text, first_name text,last_name text ,primary key (song_name,userid) )"


try:
    session.execute(query)
except Exception as e:
    print(e)     

In [33]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_history(song_name, userid , artist_name, first_name, last_name)"
        query = query + "VALUES(%s, %s, %s, %s,%s)"
        session.execute(query, (line[9],int(line[10]), line[0], line[1], line[4]))

in this query we want to get every user first name,second name in the app b   
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
by song will be primary key 

In [34]:
try:
    rows = session.execute("select first_name,last_name from user_history where song_name = 'All Hands Against His Own'")
except Exception as e:
    print(e)

    
for row in rows:
    print(row.first_name, row.last_name
         )



Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [35]:
# Drop the table before closing out the sessions

try:
    session.execute("DROP TABLE music_info_session")
except Exception as e:
    print(e)

try:
    session.execute("DROP TABLE artist_liberay ")
except Exception as e:
    print(e)
    
try:
    session.execute("DROP TABLE user_history")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [36]:
session.shutdown()
cluster.shutdown()